In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split
import requests
from tqdm.notebook import tqdm

In [2]:
df = pd.read_csv('../data/car.csv')

In [3]:
df.head()

,kind,name,image_url,start_year,end_year,doors
0,SUV cars,Audi Q3 1.4 TFSI,https://www.cars-data.com/pictures/thumbs/350p...,2013,2015,5
1,SUV cars,Audi Q5 2.0 TFSI 180hp quattro,https://www.cars-data.com/pictures/thumbs/350p...,2009,2012,5
2,SUV cars,Audi Q7 3.6 FSI quattro,https://www.cars-data.com/pictures/thumbs/350p...,2008,2008,5
3,SUV cars,Audi Q7 3.0 TFSI 272hp quattro,https://www.cars-data.com/pictures/thumbs/350p...,2010,2015,5
4,SUV cars,BMW X1 sDrive18i,https://www.cars-data.com/pictures/thumbs/350p...,2010,2011,5


In [4]:
df.drop(['name', 'start_year', 'end_year', 'doors'], axis=1, inplace=True)

In [5]:
df['kind'] = pd.factorize(df['kind'])[0]

In [6]:
df.reset_index(inplace=True)

In [7]:
y = df.pop('kind')
x = df

In [8]:
x_train, x_test, y_train, y_test = train_test_split(x,y,test_size=0.2,train_size=0.8, stratify=y, random_state=123)
x_train, x_val, y_train, y_val = train_test_split(x_train,y_train,test_size = 0.25,train_size =0.75, stratify=y_train, random_state=123)

In [9]:
x_train

,index,image_url
729,729,https://www.cars-data.com/pictures/thumbs/350p...
1887,1887,https://www.cars-data.com/pictures/thumbs/350p...
734,734,https://www.cars-data.com/pictures/thumbs/350p...
460,460,https://www.cars-data.com/pictures/thumbs/350p...
2431,2431,https://www.cars-data.com/pictures/thumbs/350p...
...,...,...
1879,1879,https://www.cars-data.com/pictures/thumbs/350p...
368,368,https://www.cars-data.com/pictures/thumbs/350p...
3482,3482,https://www.cars-data.com/pictures/thumbs/350p...
3175,3175,https://www.cars-data.com/pictures/thumbs/350p...


In [10]:
def download_pic(df, df2, folder):
    drops = []
    for idx,row in tqdm(df.iterrows(), total=df.shape[0]):
        r = requests.get(row['image_url'])
        if r.headers['Content-type'] == 'image/jpeg':
            with open(f"../data/{folder}/{row['index']}.jpg", 'wb') as f:
                f.write(r.content)
        else:
            drops.append(idx)
    return (df.drop(drops), df2.drop(drops))

In [11]:
x_train, y_train = download_pic(x_train, y_train, 'train')
x_val, y_val = download_pic(x_val, y_val, 'val')
x_test, y_test = download_pic(x_test, y_test, 'test')

  0%|          | 0/2110 [00:00<?, ?it/s]

  0%|          | 0/704 [00:00<?, ?it/s]

  0%|          | 0/704 [00:00<?, ?it/s]

In [12]:
def create_labels(x, y, folder):
    labels = pd.concat([x['index'],y],axis=1)
    labels.to_csv(f"../data/{folder}/labels.csv", index=False)

In [13]:
create_labels(x_train,y_train,'train')
create_labels(x_val,y_val,'val')
create_labels(x_test,y_test,'test')